In [1]:
from es.utils.utils import get_project_root
import os

# delete es.db
os.remove(os.path.join(get_project_root(), 'es.db'))

In [2]:
import pandas as pd
from es.utils import database_old
from es.utils import database

In [3]:
session_old = database_old.Session()
session = database.Session()

## Customers

id = Column(Integer, primary_key=True, autoincrement=True)<br>
company_name = Column(String, nullable=True)<br>
prename = Column(String, nullable=True)<br>
surname = Column(String, nullable=True)<br>
street = Column(String, nullable=False)<br>
postal_code = Column(String, nullable=False)<br>
city = Column(String, nullable=False)<br>
email = Column(String, nullable=True)<br>
phone = Column(String, nullable=True)<br>

KundenNummer: Mapped[int] = mapped_column(
    Integer, Identity(start=1, increment=1), primary_key=True
)<br>
Nachname: Mapped[Optional[str]] = mapped_column(Unicode(255))<br>
Vorname: Mapped[Optional[str]] = mapped_column(Unicode(255))<br>
Straße: Mapped[Optional[str]] = mapped_column(Unicode(255))<br>
PLZ: Mapped[Optional[str]] = mapped_column(Unicode(255))<br>
Ort: Mapped[Optional[str]] = mapped_column(Unicode(255))<br>
Land: Mapped[Optional[str]] = mapped_column(Unicode(255))<br>
Firmenname: Mapped[Optional[str]] = mapped_column(Unicode(255))<br>

In [4]:
old_customers = session_old.query(database_old.KundenÜbersicht).all()

In [5]:
for customer in old_customers:
    new_customer = database.Customer(
        id=customer.KundenNummer,
        company_name=customer.Firmenname,
        prename=customer.Vorname,
        surname=customer.Nachname,
        street=customer.Straße,
        postal_code=customer.PLZ,
        city=customer.Ort,
        country=customer.Land
    )
    session.add(new_customer)
    session.commit()

session.close()

## Shipping

id = Column(Integer, primary_key=True, autoincrement=True)<br>
name = Column(String, nullable=False)<br>
price = Column(Float, nullable=False)<br>
weight_up_to = Column(Float, nullable=True)<br>
country_class = Column(String, nullable=True)<br>
dimension_1 = Column(Float, nullable=True)<br>
dimension_2 = Column(Float, nullable=True)<br>
dimension_3 = Column(Float, nullable=True)<br>

Column("Versandart", Unicode),<br>
Column("Kosten", MONEY),<br>
Column("VersichertBis", MONEY),<br>
Column("Ländergruppe", Integer),<br>
Column("GewichtBis", Float(53)),<br>
Column("EnglischerName", Unicode),<br>
Column("Länge1", Float(53)),<br>
Column("Länge2", Float(53)),<br>
Column("Länge3", Float(53)),<br>

In [6]:
session = database.Session()

In [7]:
old_shippings = session_old.query(database_old.t_Versand).all()

In [8]:
for ship in old_shippings:
    new_ship = database.Shipping(
        name=ship.Versandart,
        name_english=ship.EnglischerName,
        insured=ship.VersichertBis,
        price=ship.Kosten,
        weight_up_to=ship.GewichtBis,
        country_class=ship.Ländergruppe,
        dimension_1=ship.Länge1,
        dimension_2=ship.Länge2,
        dimension_3=ship.Länge3,
    )
    session.add(new_ship)
    session.commit()

session.close()

## Invoice

id = Column(String, primary_key=True)<br>
customer_id = Column(Integer, ForeignKey('customers.id'))<br>
price = Column(Float, nullable=False)<br>
cash = Column(Boolean, nullable=False)<br>
invoice_date = Column(DateTime(timezone=True), nullable=False)<br>
payment_date = Column(DateTime(timezone=True), nullable=True)<br>
shipping_id = Column(Integer, ForeignKey('shipping.id'))<br>

Rechnungsnummer: Mapped[str] = mapped_column(Unicode(255), primary_key=True)<br>
Steuer: Mapped[int] = mapped_column(Integer, server_default=text("((0))"))<br>
Datum: Mapped[Optional[datetime.datetime]] = mapped_column(DATETIME2)<br>
KundenNummer: Mapped[Optional[int]] = mapped_column(
    Integer, server_default=text("((0))")
)<br>
Betrag: Mapped[Optional[Any]] = mapped_column(MONEY, server_default=text("((0))"))<br>
Versandart: Mapped[Optional[str]] = mapped_column(Unicode(255))<br>
Versandkosten: Mapped[Optional[Any]] = mapped_column(MONEY)<br>
Kasse: Mapped[Optional[bool]] = mapped_column(Boolean)<br>
Zahlungsdatum: Mapped[Optional[datetime.datetime]] = mapped_column(DATETIME2)<br>

In [9]:
session = database.Session()

In [10]:
invoices_old = session_old.query(database_old.RechnungKunde).all()

In [11]:
for inv in invoices_old:
    inv_new = database.Invoice(
        id=inv.Rechnungsnummer,
        customer_id=inv.KundenNummer,
        price=inv.Betrag,
        cash=inv.Kasse,
        invoice_date=inv.Datum,
        payment_date=inv.Zahlungsdatum,
        shipping_id=inv.Versandart,
        shipping_price=inv.Versandkosten
    )
    session.add(inv_new)
    session.commit()

session.close()